In [13]:
%defaultDatasource jdbc:h2:file:./database

In [14]:

DROP TABLE IF EXISTS DrugUseEffect;
DROP TABLE IF EXISTS Suporte;
DROP TABLE IF EXISTS Total;

DROP TABLE IF EXISTS DrugAppearance;
DROP TABLE IF EXISTS DrugaSum;
DROP TABLE IF EXISTS DrugbSum;

DROP TABLE IF EXISTS GrupoMais30;
DROP TABLE IF EXISTS Contagem;

DROP TABLE IF EXISTS Drug;
DROP TABLE IF EXISTS Pathology;
DROP TABLE IF EXISTS DrugUse;
DROP TABLE IF EXISTS SideEffect;


CREATE TABLE Drug (
  code INT NOT NULL,
  name VARCHAR(200) NOT NULL,
  PRIMARY KEY(code)
) AS SELECT
    code,
    name
FROM CSVREAD('../../../data/faers-2017/drug.csv');

CREATE TABLE Pathology (
  code INT NOT NULL,
  name VARCHAR(200) NOT NULL,
  PRIMARY KEY(code)
) AS SELECT
    code,
    name
FROM CSVREAD('../../../data/faers-2017/pathology.csv');

CREATE TABLE DrugUse (
  idPerson INT NOT NULL,
  codePathology INT NOT NULL,
  codeDrug INT NOT NULL,
  FOREIGN KEY(codePathology)
    REFERENCES Pathology(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(codeDrug)
    REFERENCES Drug(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
) AS SELECT
    idPerson,
    codePathology,
    codeDrug
FROM CSVREAD('../../../data/faers-2017/drug-use.csv');

CREATE TABLE SideEffect(
  idPerson INT NOT NULL,
  codePathology INT NOT NULL,
  FOREIGN KEY(codePathology)
    REFERENCES Pathology(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT idPerson, codePathology
FROM CSVREAD('../../../data/faers-2017/sideeffect.csv');

In [3]:
SELECT * FROM Drug;
SELECT * FROM Pathology;
SELECT * FROM DrugUse;
SELECT * FROM SideEffect;

# FDA Adverse Event Reporting System (FAERS)

Considere a base de dados que reúne relatos de efeitos adversos de drogas: [FDA Adverse Event Reporting System (FAERS)](https://open.fda.gov/data/faers/).

As fontes de dados são publicadas e documentadas em: [Latest Quarterly Data Files](https://www.fda.gov/drugs/fda-adverse-event-reporting-system-faers/fda-adverse-event-reporting-system-faers-latest-quarterly-data-files)

# Interação entre medicamentos
Considere como uma interação toda vez que um medicamento A é tomado em conjunto com um medicamento B por uma pessoa. Crie uma consulta que retorne para cada par de medicamento A,B o número de interações, somente para aqueles medicamentos que são tomados em conjunto mais de 30 vezes (passaremos a chamar de **grupo mais 30**).

In [ ]:
-- Nao consegui rodar essa parte

DROP TABLE IF EXISTS Contagem;

CREATE VIEW Contagem AS
SELECT t1.CodeDrug drug1, t2.CodeDrug drug2, COUNT(*) cont
    FROM 
        DrugUse t1, DrugUse t2
    WHERE
        t1.IdPerson = t2.IdPerson AND t1.CodeDrug < t2.CodeDrug
    GROUP BY
        t1.CodeDrug, t2.CodeDrug
;

SELECT * FROM Contagem;

In [ ]:
CREATE VIEW GrupoMais30
SELECT drug1, drug2, cont
    FROM Contagem
    WHERE cont>30
;

## Tabela Resultante

Se estiver demorando muito para gerar a tabela, você poderá usar a seguinte tabela resultante pronta. Essa tabela tem todas as interações, você ainda precisará filtrar o **grupo mais 30**.

In [4]:
DROP TABLE IF EXISTS DrugInteraction;

CREATE TABLE DrugInteraction (
  druga INT NOT NULL,
  drugb INT NOT NULL,
  weight INT NOT NULL,
  PRIMARY KEY(druga, drugb)
) AS SELECT
    druga,
    drugb,
    weight
FROM CSVREAD('../../../data/faers-2017/results/drug-interaction.csv');

SELECT * FROM DrugInteraction;



In [5]:
DROP TABLE IF EXISTS GrupoMais30;

CREATE VIEW GrupoMais30 AS
SELECT druga, drugb, weight
    FROM DrugInteraction
    WHERE weight > 30
;

SELECT * FROM GrupoMais30;

# Suporte

### Qual o suporte que cada uma das associações tem?

Gere uma consulta que apresente o suporte do **grupo mais 30**. Ordene o resultado por ordem de suporte. 

Observação: Para que o resultado da divisão seja fracionária, converta o numerador em `DOUBLE` usando:
~~~sql
CAST(campo AS DOUBLE)
~~~

In [6]:
DROP TABLE IF EXISTS Suporte;
DROP TABLE IF EXISTS Total;

CREATE VIEW Total AS
SELECT SUM(Weight) Total
    FROM DrugInteraction
;

SELECT * FROM Total;

CREATE VIEW Suporte AS
SELECT g.druga, g.drugb, g.weight, CAST(g.weight AS DOUBLE)/t.total suporte
    FROM GrupoMais30 g, Total t
    ORDER BY suporte
;

SELECT * FROM Suporte;

# Confiança

### Qual a confiança que cada uma das associações tem?

Gere uma consulta que apresente a confiança do **grupo mais 30**. Ordene o resultado por ordem de confiança.

In [7]:
DROP TABLE IF EXISTS DrugAppearance;
DROP TABLE IF EXISTS DrugaSum;
DROP TABLE IF EXISTS DrugbSum;

-- drug sum
CREATE VIEW DrugaSum AS
SELECT druga, SUM(Weight) drugaTotal
    FROM DrugInteraction
    GROUP BY druga
;

CREATE VIEW DrugbSum AS
SELECT drugb, SUM(Weight) drugbTotal
    FROM DrugInteraction
    GROUP BY drugb
;

CREATE VIEW DrugAppearance AS
SELECT a.druga drug, a.drugatotal, b.drugbtotal,  a.drugatotal + b.drugbtotal appearances
    FROM DrugaSum a, DrugbSum b
    WHERE a.druga = b.drugb
;


SELECT g.druga, g.drugb, g.weight, da.appearances, CAST(g.weight AS DOUBLE)/(da.appearances - g.weight) confianca
    FROM GrupoMais30 g, DrugAppearance da
    WHERE g.druga = da.drug
    ORDER BY Confianca
;

SELECT g.drugb, g.druga, g.weight, da.appearances, CAST(g.weight AS DOUBLE)/(da.appearances - g.weight) confianca
    FROM GrupoMais30 g, DrugAppearance da
    WHERE g.drugb = da.drug
    ORDER BY Confianca
;

# Efeito Colateral

Para cada medicamento, qual o percentual do medicamento A que tem efeito colateral junto com o medicamento B, quando comparado com seu uso sem o medicamento B.

In [22]:
-- Nao consegui rodar essa parte

-- Juntar uso de medicamento e efeitos
-- 19919
DROP TABLE IF EXISTS DrugUseEffect;

CREATE VIEW DrugUseEffect AS
SELECT du.IdPerson, du.CodeDrug
    FROM DrugUse du, SideEffect se
    WHERE du.IdPerson = se.IdPerson
    GROUP BY du.IdPerson, du.CodeDrug
    ORDER BY du.IdPerson
;

In [23]:
-- contagem interacoes com efeito colateral

DROP TABLE IF EXISTS Contagem;

CREATE VIEW Contagem AS
SELECT t1.CodeDrug druga, t2.CodeDrug drugb, COUNT(*) weight
    FROM 
        DrugUseEffect t1, DrugUseEffect t2
    WHERE
        t1.IdPerson = t2.IdPerson AND t1.CodeDrug < t2.CodeDrug 
    GROUP BY
        t1.CodeDrug, t2.CodeDrug
;


In [24]:
DROP TABLE IF EXISTS DrugAppearance;
DROP TABLE IF EXISTS DrugSum;

-- soma do uso da droga com efeito colateral
CREATE VIEW DrugSum AS
SELECT due.CodeDrug, COUNT(*) weight
    FROM DrugUseEffect due
    GROUP BY due.codedrug
;



In [ ]:
SELECT g.druga, g.drugb, g.weight, da.weight, CAST(g.weight AS DOUBLE)/(da.weight - g.weight) proporcao
    FROM Contagem g, DrugSum da
    WHERE g.druga = da.CodeDrug
    ORDER BY proporcao
;